Importing pandas ; Importing price prediction by book value growth csv ; Importing the price csv

In [22]:
import pandas as pd
df_prediction=pd.read_csv('../../data/output/price-prediction-by-book-value-growth.csv',sep=',')
df_price=pd.read_csv('../../data/input/price.csv',sep=',')

Dropping the prediction columns except the 3 Prediction column 

In [23]:
df_prediction=df_prediction[['Year','3 Prediction']]
df_prediction=df_prediction.set_index('Year')
print(df_prediction.head(len(df_prediction)))

      3 Prediction
Year              
2014   2369.029813
2015   3225.604601
2016   3191.906402
2017   3746.859389
2018   5218.683060
2019   6211.703465
2020   7741.498318
2021   7587.731096
2022   8046.755950
2023   9262.144589
2024  10290.176731


Creating the price year index ; Calculating price means by month and year

In [24]:
df_price=df_price.drop(columns=['Open','High','Low','Volume','Adj Close'])

df_price['Year']=pd.to_datetime(df_price['Date']).dt.year
df_price=df_price.set_index('Year')
df_price['Month']=pd.to_datetime(df_price['Date']).dt.month
df_price=df_price.drop(columns=['Date'])

df_price=df_price.groupby(['Year','Month']).mean()
print(df_price[df_price.index.get_level_values('Year')==2017])

                  Close
Year Month             
2017 1      3069.772727
     2      3097.000000
     3      3193.409091
     4      3453.333333
     5      3539.500000
     6      3582.954545
     7      3696.190476
     8      3775.652174
     9      3844.761905
     10     4104.090909
     11     4197.727273
     12     4238.809524


Creating the progress dataframe

In [25]:
df_progress=df_price
print(df_progress[df_progress.index.get_level_values('Year')==2017])
df_progress['Progress']=df_progress['Close']/df_prediction['3 Prediction']

print(df_progress[df_progress.index.get_level_values('Year')==2017])
# make month index available as column
df_progress['Month']=[i[1] for i in df_progress.index]

                  Close
Year Month             
2017 1      3069.772727
     2      3097.000000
     3      3193.409091
     4      3453.333333
     5      3539.500000
     6      3582.954545
     7      3696.190476
     8      3775.652174
     9      3844.761905
     10     4104.090909
     11     4197.727273
     12     4238.809524
                  Close  Progress
Year Month                       
2017 1      3069.772727  0.819292
     2      3097.000000  0.826559
     3      3193.409091  0.852290
     4      3453.333333  0.921661
     5      3539.500000  0.944658
     6      3582.954545  0.956255
     7      3696.190476  0.986477
     8      3775.652174  1.007685
     9      3844.761905  1.026129
     10     4104.090909  1.095342
     11     4197.727273  1.120332
     12     4238.809524  1.131297


Calculating lower bound and upper bound for progress dataframe cleansing

In [26]:
months=range(1,13)
lower_bound = []
upper_bound = []

for month in months:
    q1 = df_progress[df_progress['Month'] == month]['Progress'].quantile(0.25)
    q3 = df_progress[df_progress['Month'] == month]['Progress'].quantile(0.75)
    iqr = q3 - q1
    lower_bound.append(q1 - 1.5 * iqr)
    upper_bound.append(q3 + 1.5 * iqr)  

Cleaning the progress dataframe from outliers

In [27]:
df_cleaned_progress=pd.DataFrame()
for month in months:
    df_cleaned_progress=pd.concat([df_cleaned_progress,df_progress[(df_progress['Month'] == month) & (df_progress['Progress'] > lower_bound[month-1]) & (df_progress['Progress'] < upper_bound[month-1])]])

# dropping the month column
df_cleaned_progress=df_cleaned_progress.drop(columns=['Month','Close'])
df_cleaned_progress=df_cleaned_progress.sort_values(by=['Year','Month'])

print(df_cleaned_progress.head(len(df_cleaned_progress)))

            Progress
Year Month          
2014 1      0.825127
     2      0.864067
     3      0.887600
     4      0.928349
     5      0.943307
...              ...
2023 12     0.978090
2024 1      0.930610
     2      0.954254
     4      0.940976
     5      0.938273

[120 rows x 1 columns]


Exporting the progress dataframe csv

In [28]:
df_cleaned_progress.to_csv('../../data/output/monthly-progress-by-book-value/mean.csv',sep=';',index=True)